1. Bibliotecas 

In [16]:
#!pip install streamlit

In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st

2. Carregando os registros

In [12]:
filepath = filepath = "C:\\Users\\joaom\\OneDrive\\Trades e investimentos\\Swing trades\\REGISTROS.xlsx"
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
diarioccm = pd.read_excel(filepath, sheet_name='CCMFUT')
diariobgi = pd.read_excel(filepath, sheet_name='BGIFUT')
diarioacoes = pd.read_excel(filepath, sheet_name='ACOES')
diariofull = pd.concat([diarioccm, diariobgi, diarioacoes]) #diariofull = com as operações em aberto
diariofull.sort_values(by='SAÍDA', inplace=True)

3. Calculando as métricas

In [13]:
diariofull['POS E'] = diariofull['PREÇO ENTRADA'] * diariofull['QDE']
diariofull['POS S'] = diariofull['PREÇO SAIDA'] * diariofull['QDE']
# Calcula a coluna 'VAR' com base na direção da negociação
diariofull['VAR'] = np.where(
    diariofull['DIREÇÃO'] == 'LONG',
    diariofull['PREÇO SAIDA'] - diariofull['PREÇO ENTRADA'],
    diariofull['PREÇO ENTRADA'] - diariofull['PREÇO SAIDA'])
# Variação Percentual de cada trade em relação ao preço de entrada
diariofull['VAR%'] = (diariofull['VAR'] / diariofull['PREÇO ENTRADA']) * 100
# Lucro Bruto dos trades
diariofull['GROSS P/L'] = (diariofull['VAR'] * diariofull['QDE'])
# Lucro líquido (Apenas custos operacionais)
diariofull['NET P/L'] = (diariofull['GROSS P/L'] - diariofull['COSTS'])
# P/L Acumulado
diariofull['P/L ACUM'] = diariofull['NET P/L'].cumsum()
# VAR% Acumulado 
diariofull['VAR% ACUM'] = diariofull['VAR%'].cumsum()
# Equity
diariofull['AJUSTES'].fillna(0, inplace=True)
diariofull["EQUITY"] = diariofull['P/L ACUM'] + diariofull['AJUSTES'].cumsum()
# Peak (valor máximo, para cálculo de drawdown)
diariofull['PEAK'] = diariofull['P/L ACUM'].cummax()
# EQUITY Drawdown
diariofull['EDD'] = np.where(
    diariofull['VAR'] < 0,
    diariofull['P/L ACUM'] - diariofull['PEAK'],
    0
)
# Equity Drawdown %%
diariofull['EDD%'] = (diariofull['EDD'] / diariofull['EQUITY']) * 100

In [14]:
diariofull

,ENTRADA,SAÍDA,ATIVO,DIREÇÃO,SETUP,PREÇO ENTRADA,QDE,STOP,PREÇO SAIDA,COSTS,AJUSTES,POS E,POS S,VAR,VAR%,GROSS P/L,NET P/L,P/L ACUM,VAR% ACUM,EQUITY,PEAK,EDD,EDD%
0,2021-06-04,2021-06-14,CCMFUT,LONG,MM8,97.25,450.0,NaN,92.42,30,15000.0,43762.5,41589.0,-4.83,-4.966581,-2173.5,-2203.5,-2203.5,-4.966581,12796.5,-2203.5,0.0,0.000000
1,2021-06-14,2021-06-25,CCMFUT,SHORT,MM8,92.42,450.0,NaN,83.52,30,0.0,41589.0,37584.0,8.90,9.629950,4005.0,3975.0,1771.5,4.663369,16771.5,1771.5,0.0,0.000000
2,2021-06-25,2021-06-25,CCMFUT,SHORT,MM8,83.84,450.0,NaN,84.70,30,0.0,37728.0,38115.0,-0.86,-1.025763,-387.0,-417.0,1354.5,3.637606,16354.5,1771.5,-417.0,-2.549757
3,2021-06-25,2021-07-08,CCMFUT,LONG,MM8,84.70,450.0,NaN,97.90,30,0.0,38115.0,44055.0,13.20,15.584416,5940.0,5910.0,7264.5,19.222021,22264.5,7264.5,0.0,0.000000
4,2021-07-08,2021-07-08,CCMFUT,SHORT,MM8,96.80,450.0,NaN,97.95,30,0.0,43560.0,44077.5,-1.15,-1.188017,-517.5,-547.5,6717.0,18.034005,21717.0,7264.5,-547.5,-2.521066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,2024-01-24,NaT,CCMFUT,LONG,MM8,66.57,2250.0,NaN,NaN,20,0.0,149782.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
12,2024-01-12,NaT,BGIFUT,SHORT,MM8,245.35,330.0,NaN,NaN,8,0.0,80965.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
35,2024-01-18,NaT,TOTS3,SHORT,R1,31.49,500.0,13.42,NaN,14,0.0,15745.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
36,2024-01-22,NaT,ITUB4,SHORT,R1,27.45,500.0,33.29,NaN,14,0.0,13725.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


4. Criação do Dashboard usando a Streamlit

SyntaxError: invalid syntax (1151618118.py, line 1)